In [ ]:
import pyspark
import pyspark.sql.functions as F
import json

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
metrics = spark.read.json("metrics/application_1601392010735_0030")

In [ ]:
def collect_and_dictify(df):
    return [json.loads(row[0]) for row in df.selectExpr("to_json(*)").collect()]

def executor_info(df):
    info = df.select("Executor Info").dropna()
    return collect_and_dictify(info)

In [ ]:
def plan_info(df):
    return collect_and_dictify(df.select("sparkPlanInfo").dropna())

In [ ]:
def stageInfo(df):
    
    return collect_and_dictify(df.select("Stage Info").dropna())

In [ ]:
def wide_metrics(df, mcol='Task Info', idcol='Task ID'):
    acc_cols = [F.col('Accumulable.%s' % s).alias('Metric %s' % s) for s in ['ID', 'Name', 'Value']]
    obs = df.select(mcol).dropna().select('%s.*' % mcol)
    cols = [F.col(elt) for elt in sorted(set(obs.columns) - set([idcol, 'Accumulables']))]
    return obs.select(
        idcol, 
        F.explode('Accumulables').alias('Accumulable'), 
        *cols
    ).select(idcol, *(cols + acc_cols))

def wide_tasks(df):
    return wide_metrics(df, 'Task Info', 'Task ID')

def wide_stages(df):
    return wide_metrics(df, 'Stage Info', 'Stage ID')


In [ ]:
task_metrics = wide_tasks(metrics).toPandas()